# 1 序贯（Sequential）模型
序贯模型是多个网络层的线性堆叠，也就是“一条路走到黑”。
可以通过向 Sequential 模型传递一个 layer 的 `list` 来构造该模型：

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential([Dense(32, input_shape=(784,)),
                    Activation('relu'),
                    Dense(10),
                    Activation('softmax'),
                   ])

也可以通过 `.add()` 方法一个个的将 layer 加入模型中：

In [11]:
model = Sequential()
model.add(Dense(32, input_shape=(784,)))
model.add(Activation('relu'))

## 1.1 指定输入数据的 shape
模型需要知道输入数据的shape，因此，`Sequential` 的第一层需要接受一个关于输入数据 shape 的参数，后面的各个层则可以自动的推导出中间数据的shape，因此不需要为每个层都指定这个参数。有几种方法来为第一层指定输入数据的 shape：
- 传递一个`input_shape`的关键字参数给第一层，`input_shape`是一个 tuple 类型的数据，其中也可以填入 `None` ，如果填入 `None` 则表示此位置可能是任何正整数。数据的 `batch` 大小不应包含在其中。
- 有些2D层，如 `Dense`，支持通过指定其输入维度 `input_dim` 来隐含的指定输入数据 shape,是一个 Int 类型的数据。一些 3D 的时域层支持通过参数`input_dim` 和`input_length` 来指定输入 shape。
- 如果你需要为输入指定一个固定大小的 `batch_size`（常用于stateful RNN网络），可以传递 `batch_size` 参数到一个层中，例如你想指定输入张量的`batch` 大小是$32$，数据shape是$(6，8)$，则你需要传递 `batch_size=32` 和 `input_shape=(6,8)`。

In [12]:
model = Sequential()
model.add(Dense(32, input_dim=784))

In [13]:
model = Sequential()
model.add(Dense(32, input_shape=(784,)))

## 1.2 编译
在训练模型之前，我们需要通过 `compile` 来对学习过程进行配置。
`compile`接收三个参数：
- 优化器 `optimizer`：该参数可指定为已预定义的优化器名，如 `rmsprop、adagrad` ，或一个`Optimizer` 类的对象，详情见[优化器optimizers](http://keras-cn.readthedocs.io/en/latest/other/optimizers/)。
- 损失函数 `loss`：该参数为模型试图最小化的目标函数，它可为预定义的损失函数名，如 `categorical_crossentropy、mse`，也可以为一个自定义损失函数。详情见[损失函数loss ](http://keras-cn.readthedocs.io/en/latest/other/objectives/)。
- 指标列表 `metrics`：对分类问题，我们一般将该列表设置为 `metrics=['accuracy']`。指标可以是一个预定义指标的名字,也可以是一个用户定制的函数.指标函数应该返回单个张量,或一个完成 `metric_name - > metric_value`映射的字典.

In [14]:
# For a multi-class classification problem
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# For a binary classification problem
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# For a mean squared error regression problem
model.compile(optimizer='rmsprop',
              loss='mse')

# For custom metrics
import keras.backend as K

def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy', mean_pred])

## 1.3 训练
Keras以Numpy数组作为输入数据和标签的数据类型。训练模型一般使用fit函数，

In [15]:
# For a single-input model with 2 classes (binary classification):

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=100))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Generate dummy data
import numpy as np
data = np.random.random((1000, 100))
labels = np.random.randint(2, size=(1000, 1))

# Train the model, iterating on the data in batches of 32 samples
model.fit(data, labels, epochs=10, batch_size=32)
# For a single-input model with 10 classes (categorical classification):

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=100))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Generate dummy data
import numpy as np
data = np.random.random((1000, 100))
labels = np.random.randint(10, size=(1000, 1))

# Convert labels to categorical one-hot encoding
one_hot_labels = keras.utils.to_categorical(labels, num_classes=10)

# Train the model, iterating on the data in batches of 32 samples
model.fit(data, one_hot_labels, epochs=10, batch_size=32)

Epoch 1/10
1000/1000 [==============================] - 5s 5ms/step - loss: 0.7077 - acc: 0.5080
Epoch 2/10
1000/1000 [==============================] - 0s 396us/step - loss: 0.7023 - acc: 0.5350
Epoch 3/10
1000/1000 [==============================] - 0s 141us/step - loss: 0.6901 - acc: 0.5410
Epoch 4/10
1000/1000 [==============================] - 0s 138us/step - loss: 0.6867 - acc: 0.5570
Epoch 5/10
1000/1000 [==============================] - 0s 205us/step - loss: 0.6790 - acc: 0.5820
Epoch 6/10
1000/1000 [==============================] - 0s 228us/step - loss: 0.6739 - acc: 0.5880
Epoch 7/10
1000/1000 [==============================] - 0s 242us/step - loss: 0.6678 - acc: 0.5990
Epoch 8/10
1000/1000 [==============================] - 0s 186us/step - loss: 0.6668 - acc: 0.6040
Epoch 9/10
1000/1000 [==============================] - 0s 141us/step - loss: 0.6624 - acc: 0.6110
Epoch 10/10
1000/1000 [==============================] - 0s 183us/step - loss: 0.6572 - acc: 0.6070


NameError: name 'keras' is not defined

### 基于多层感知器的softmax多分类：

In [19]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

# Generate dummy data
import numpy as np
x_train = np.random.random((1000, 20))
y_train = keras.utils.to_categorical(np.random.randint(10, size=(1000, 1)), num_classes=10)
x_test = np.random.random((100, 20))
y_test = keras.utils.to_categorical(np.random.randint(10, size=(100, 1)), num_classes=10)

model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
model.add(Dense(64, activation='relu', input_dim=20))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(x_train, y_train,
          epochs=20,
          batch_size=128)
score = model.evaluate(x_test, y_test, batch_size=128)

Epoch 1/20
1000/1000 [==============================] - 1s 919us/step - loss: 2.4567 - acc: 0.1020
Epoch 2/20
1000/1000 [==============================] - 0s 55us/step - loss: 2.3691 - acc: 0.1130
Epoch 3/20
1000/1000 [==============================] - 0s 58us/step - loss: 2.3450 - acc: 0.1080
Epoch 4/20
1000/1000 [==============================] - 0s 65us/step - loss: 2.3284 - acc: 0.1060
Epoch 5/20
1000/1000 [==============================] - 0s 62us/step - loss: 2.3391 - acc: 0.1020
Epoch 6/20
1000/1000 [==============================] - 0s 63us/step - loss: 2.3201 - acc: 0.1130
Epoch 7/20
1000/1000 [==============================] - 0s 63us/step - loss: 2.3103 - acc: 0.1060
Epoch 8/20
1000/1000 [==============================] - 0s 56us/step - loss: 2.3126 - acc: 0.0970
Epoch 9/20
1000/1000 [==============================] - 0s 58us/step - loss: 2.3152 - acc: 0.1010
Epoch 10/20
1000/1000 [==============================] - 0s 56us/step - loss: 2.3025 - acc: 0.1290
Epoch 11/20
1000/1

### MLP的二分类：

In [20]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout

# Generate dummy data
x_train = np.random.random((1000, 20))
y_train = np.random.randint(2, size=(1000, 1))
x_test = np.random.random((100, 20))
y_test = np.random.randint(2, size=(100, 1))

model = Sequential()
model.add(Dense(64, input_dim=20, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model.fit(x_train, y_train,
          epochs=20,
          batch_size=128)
score = model.evaluate(x_test, y_test, batch_size=128)

Epoch 1/20
1000/1000 [==============================] - 1s 862us/step - loss: 0.7224 - acc: 0.4840
Epoch 2/20
1000/1000 [==============================] - 0s 51us/step - loss: 0.7030 - acc: 0.5140
Epoch 3/20
1000/1000 [==============================] - 0s 55us/step - loss: 0.7098 - acc: 0.4710
Epoch 4/20
1000/1000 [==============================] - 0s 53us/step - loss: 0.6991 - acc: 0.5080
Epoch 5/20
1000/1000 [==============================] - 0s 51us/step - loss: 0.7090 - acc: 0.4850
Epoch 6/20
1000/1000 [==============================] - 0s 50us/step - loss: 0.6946 - acc: 0.5140
Epoch 7/20
1000/1000 [==============================] - 0s 53us/step - loss: 0.6970 - acc: 0.5320
Epoch 8/20
1000/1000 [==============================] - 0s 51us/step - loss: 0.6950 - acc: 0.5070
Epoch 9/20
1000/1000 [==============================] - 0s 53us/step - loss: 0.6975 - acc: 0.5090
Epoch 10/20
1000/1000 [==============================] - 0s 55us/step - loss: 0.6955 - acc: 0.5060
Epoch 11/20
1000/1

### 类似VGG的卷积神经网络：

In [21]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD

# Generate dummy data
x_train = np.random.random((100, 100, 100, 3))
y_train = keras.utils.to_categorical(np.random.randint(10, size=(100, 1)), num_classes=10)
x_test = np.random.random((20, 100, 100, 3))
y_test = keras.utils.to_categorical(np.random.randint(10, size=(20, 1)), num_classes=10)

model = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)

model.fit(x_train, y_train, batch_size=32, epochs=10)
score = model.evaluate(x_test, y_test, batch_size=32)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/10
100/100 [==============================] - 4s 43ms/step - loss: 2.3713
Epoch 2/10
100/100 [==============================] - 1s 6ms/step - loss: 2.3049
Epoch 3/10
100/100 [==============================] - 1s 6ms/step - loss: 2.3083
Epoch 4/10
100/100 [==============================] - 1s 6ms/step - loss: 2.2827
Epoch 5/10
100/100 [==============================] - 1s 6ms/step - loss: 2.2745
Epoch 6/10
100/100 [==============================] - 1s 6ms/step - loss: 2.2948
Epoch 7/10
100/100 [==============================] - 1s 6ms/step - loss: 2.3193
Epoch 8/10
100/100 [==============================] - 1s 6ms/step - loss: 2.2937
Epoch 9/10
100/100 [==============================] - 1s 6ms/step - loss: 2.2816
Epoch 10/10
20/20 [==============================] - 0s 14ms/step


### 使用LSTM的序列分类

In [24]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM

model = Sequential()
model.add(Embedding(max_features, output_dim=256))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=16, epochs=10)
score = model.evaluate(x_test, y_test, batch_size=16)

NameError: name 'max_features' is not defined

### 使用1D卷积的序列分类

In [23]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D

model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=(seq_length, 100)))
model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=16, epochs=10)
score = model.evaluate(x_test, y_test, batch_size=16)

NameError: name 'seq_length' is not defined

### 用于序列分类的栈式LSTM
在该模型中，我们将三个LSTM堆叠在一起，是该模型能够学习更高层次的时域特征表示。
开始的两层LSTM返回其全部输出序列，而第三层LSTM只返回其输出序列的最后一步结果，从而其时域维度降低（即将输入序列转换为单个向量）

In [25]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

data_dim = 16
timesteps = 8
num_classes = 10

# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(32, return_sequences=True,
               input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32))  # return a single vector of dimension 32
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Generate dummy training data
x_train = np.random.random((1000, timesteps, data_dim))
y_train = np.random.random((1000, num_classes))

# Generate dummy validation data
x_val = np.random.random((100, timesteps, data_dim))
y_val = np.random.random((100, num_classes))

model.fit(x_train, y_train,
          batch_size=64, epochs=5,
          validation_data=(x_val, y_val))

Train on 1000 samples, validate on 100 samples
Epoch 1/5
1000/1000 [==============================] - 4s 4ms/step - loss: 11.5130 - acc: 0.1190 - val_loss: 11.1773 - val_acc: 0.1200
Epoch 2/5
1000/1000 [==============================] - 1s 683us/step - loss: 11.5117 - acc: 0.1090 - val_loss: 11.1774 - val_acc: 0.1200
Epoch 3/5
1000/1000 [==============================] - 1s 685us/step - loss: 11.5116 - acc: 0.1030 - val_loss: 11.1744 - val_acc: 0.1200
Epoch 4/5
1000/1000 [==============================] - 1s 691us/step - loss: 11.5111 - acc: 0.1040 - val_loss: 11.1764 - val_acc: 0.0900
Epoch 5/5
1000/1000 [==============================] - 1s 682us/step - loss: 11.5112 - acc: 0.1030 - val_loss: 11.1766 - val_acc: 0.1100


### 采用stateful LSTM的相同模型
stateful LSTM的特点是，在处理过一个batch的训练数据后，其内部状态（记忆）会被作为下一个batch的训练数据的初始状态。状态LSTM使得我们可以在合理的计算复杂度内处理较长序列

In [26]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

data_dim = 16
timesteps = 8
num_classes = 10
batch_size = 32

# Expected input batch shape: (batch_size, timesteps, data_dim)
# Note that we have to provide the full batch_input_shape since the network is stateful.
# the sample of index i in batch k is the follow-up for the sample i in batch k-1.
model = Sequential()
model.add(LSTM(32, return_sequences=True, stateful=True,
               batch_input_shape=(batch_size, timesteps, data_dim)))
model.add(LSTM(32, return_sequences=True, stateful=True))
model.add(LSTM(32, stateful=True))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Generate dummy training data
x_train = np.random.random((batch_size * 10, timesteps, data_dim))
y_train = np.random.random((batch_size * 10, num_classes))

# Generate dummy validation data
x_val = np.random.random((batch_size * 3, timesteps, data_dim))
y_val = np.random.random((batch_size * 3, num_classes))

model.fit(x_train, y_train,
          batch_size=batch_size, epochs=5, shuffle=False,
          validation_data=(x_val, y_val))

Train on 320 samples, validate on 96 samples
Epoch 1/5
320/320 [==============================] - 3s 10ms/step - loss: 11.5195 - acc: 0.0969 - val_loss: 11.4322 - val_acc: 0.0833
Epoch 2/5
320/320 [==============================] - 0s 1ms/step - loss: 11.5146 - acc: 0.1094 - val_loss: 11.4324 - val_acc: 0.0938
Epoch 3/5
320/320 [==============================] - 0s 1ms/step - loss: 11.5138 - acc: 0.1094 - val_loss: 11.4324 - val_acc: 0.0938
Epoch 4/5
320/320 [==============================] - 0s 1ms/step - loss: 11.5132 - acc: 0.1094 - val_loss: 11.4326 - val_acc: 0.0938
Epoch 5/5
320/320 [==============================] - 0s 1ms/step - loss: 11.5126 - acc: 0.1031 - val_loss: 11.4328 - val_acc: 0.0833


# Keras FAQ：
常见问题: http://keras-cn.readthedocs.io/en/latest/for_beginners/FAQ/